In [1]:
import pandas as pd

In [2]:
covid_df = pd.read_csv('COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv')

C:\Users\dhh3hb\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
covid_df[covid_df['res_state'] == 'MO'].head()

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
11265,2020-07,MO,29.0,GREENE,29077.0,NaN,NaN,NaN,NaN,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,Missing,NaN
11266,2020-07,MO,29.0,GREENE,29077.0,NaN,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,NaN
11267,2020-07,MO,29.0,GREENE,29077.0,NaN,NaN,NaN,NaN,NaN,NaN,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,NaN
11268,2020-07,MO,29.0,GREENE,29077.0,NaN,NaN,NaN,NaN,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,Missing,NaN
11269,2020-07,MO,29.0,GREENE,29077.0,NaN,NaN,NaN,NaN,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,Missing,NaN
